In [1]:
import h5py
import pylab as pl
import numpy as np
import scipy.stats as sp
import matplotlib as mpl
import matplotlib.colors as mpc
import networkx
import progressbar
import astropy.cosmology as apc

In [2]:
def split_unique_id(unique_id):
    """Splits the ids assign to the subhalos by the merger tree code by snap number and subfind number """
    subfind_number = int(unique_id % 1e6)
    snap_number = int((unique_id - subfind_number) / 1e6)
    
    return snap_number, subfind_number

def get_main_branch_unique_ids(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    i = 0
    while True:
        succesors = list(subtree.successors(node))
        if len(succesors) == 0:
            break
        node = succesors[0] # select only the first succesor (main branch)
        mpb.append(node)
        
    return mpb

In [3]:
pl.rc('font', size = 20)
pl.rc('legend', fontsize = 20)
sim = h5py.File('/disk1/data1-casona/data1/cielo/simulations/LG1/LG1.hdf5', 'r')
trees = networkx.read_multiline_adjlist('/disk1/data1-casona/data1/cielo/simulations/LG1/LG1_tree.dat')

In [4]:
Slist = h5py.File('SatelliteList_V2_withAngles_n500_KoK04_PairsV2.h5', 'r')
Satellites = Slist['SatelliteList'][()]
Centrals = Slist['SatelliteCentral'][()]
inPairs = Slist['inGroup']
Mu = Slist['Mu']

In [5]:
# first I need the snap and id of the firts time that the group is at r=1/r_200

Snap, idSnap, cenSnap = [], [], []

for s, c in zip(Satellites, Centrals):
    uniqueID = int(127*1e6+s)
    stree = networkx.dfs_tree(trees, str(uniqueID))
    mtree = get_main_branch_unique_ids(stree, str(uniqueID))
    
    # and the centre
    uniqueIDC = int(127*1e6+c)
    streeC = networkx.dfs_tree(trees, str(uniqueIDC))
    mtreeC = get_main_branch_unique_ids(streeC, str(uniqueIDC))
    
    thesnap, thegal, thecentre = 127, s, c
    for m, mC in zip(mtree, mtreeC):
        snap, ffT = split_unique_id(int(m))
        snapC, ccT = split_unique_id(int(mC))
        
        gro = sim['SnapNumber_{}/SubGroups/GroupNumber'.format(snap)][ccT]
        R200 = sim['SnapNumber_{}/Groups/Group_R_Crit200'.format(snap)][gro]
        
        cen = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][ccT]
        gal = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][ffT]
        
        dist = np.sqrt(np.sum((cen-gal)**2))/R200
        
        
        if dist<1.:
            thesnap = snap
            thegal = ffT
            thecentre = ccT
            
    Snap.append(thesnap)
    idSnap.append(thegal)
    cenSnap.append(thecentre)

Snap, idSnap, cenSnap = np.array(Snap), np.array(idSnap), np.array(cenSnap)

In [6]:
Snap

array([127, 109,  90, 108,  86, 120,  99, 117, 102, 127, 127])

In [7]:
# and now lets take into account the time variation, lets see.

def getAnglesTime(gal, centre, snap):
    RotMass = sim['SnapNumber_{}/SubGroups/PartType4/RotationMatrix'.format(snap)][gal]
    cenGas = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][gal]
    RotII = RotMass.T
    R = np.array([0., 0., 1.])
    R_rot = np.dot(RotII, R.T).T
    
    velGal = sim['SnapNumber_{}/SubGroups/SubGroupVel'.format(snap)][gal]
    cenGro = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][centre]
    velGro = sim['SnapNumber_{}/SubGroups/SubGroupVel'.format(snap)][centre]
    
    vecCen = cenGro - cenGas
    vecVel = velGal - velGro
    
    vecCenN = vecCen/np.sqrt(np.sum(vecCen**2.))
    vecVelN = vecVel/np.sqrt(np.sum(vecVel**2.))
    
    # and cosines
    cosCen = np.dot(R_rot, vecCenN)
    cosVel = np.dot(R_rot, vecVelN)
        
    return cosCen, cosVel

In [8]:
CosCen, CosVel = [], []

for sn, idd, cen in zip(Snap, idSnap, cenSnap):
#     cosCen, cosVel = getAnglesTime(idd, cen, sn)
    cosCen, cosVel = 0.,0.
    CosCen.append(cosCen)
    CosVel.append(cosVel)
    
CosCen, CosVel = np.array(CosCen), np.array(CosVel)

In [9]:
# and stellar mass at z=0
StMass = []

for s in Satellites:
    offStar = sim['SnapNumber_128/SubGroups/PartType4/Offsets'][s].astype('int')
    massStar = sim['SnapNumber_128/PartType4/Masses'][offStar[0]:offStar[1]]
    StMass.append(np.log10(np.sum(massStar)*1e10))
    
StMass = np.array(StMass)

In [10]:
# with h5py.File('SatelliteList_V2_withAngles_n500_KoK04_All.h5', 'w') as f:
#     f.create_dataset('SatelliteList', data=Satellites)
#     f.create_dataset('SatelliteCentral', data=Centrals)
#     f.create_dataset('inGroup', data=inPairs)
#     f.create_dataset('cosCen', data=CosCen)
#     f.create_dataset('cosVel', data=CosVel)
#     f.create_dataset('StMass', data=StMass)
#     f.create_dataset('Mu', data=Mu)

In [11]:
print('IDs\tCent\tinPairs\tcosCen\t\t\tcosVel\t\t\tstMass\t\t\tMu')
for i in range(Satellites.size):
    print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(Satellites[i], Centrals[i], inPairs[i], CosCen[i], CosVel[i], StMass[i], Mu[i]))

IDs	Cent	inPairs	cosCen			cosVel			stMass			Mu
152	151	True	0.0	0.0	7.69437017371166	0.19028741328047571
4338	4337	True	0.0	0.0	8.824296879623581	0.5248884311706145
4339	4337	True	0.0	0.0	8.395722114289644	2.086676217765043
4341	4337	True	0.0	0.0	8.237350760201767	2.198490566037736
4343	4337	False	0.0	0.0	8.100917132543179	0.0
4470	4469	True	0.0	0.0	9.557141203991026	0.446042746491177
4471	4469	True	0.0	0.0	9.561488709317695	3.0531650071123755
4473	4469	True	0.0	0.0	8.581329133791673	4.319748427672956
4474	4469	True	0.0	0.0	8.71543015412092	11.480499219968799
4476	4469	True	0.0	0.0	8.267004492441973	25.38836865450961
4478	4469	False	0.0	0.0	8.021852922806016	0.0


In [12]:
1./0.446042746491177

2.2419375897636766

In [13]:
np.cos(np.pi/4.)

0.7071067811865476

In [14]:
# Ok, history of mergers for each galaxy, but firts for one galaxy.
uid = int(127*1e6+4338)
stree = networkx.dfs_tree(trees, str(uid))

uidC = int(127*1e6+4337)
streeC = networkx.dfs_tree(trees, str(uidC))

Mergers, distance = [], []

while True:
    successors = list(stree.successors(str(uid)))
    successorsC = list(streeC.successors(str(uidC)))
    if len(successors) == 0:
        break
    uid = successors[0]
    uidC = successorsC[0]
    if len(successors)>1: # that is, a mergers happens
        snap, ffT = split_unique_id(int(uid))
        snapC, ccT = split_unique_id(int(uidC))
        
        gro = sim['SnapNumber_{}/SubGroups/GroupNumber'.format(snap)][ccT]
        R200 = sim['SnapNumber_{}/Groups/Group_R_Crit200'.format(snap)][gro]
        
        cen = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][ccT]
        gal = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][ffT]
        
        dist = np.sqrt(np.sum((cen-gal)**2))/R200
        
        distance.append(dist)
        
        offMain = sim['SnapNumber_{}/SubGroups/PartType4/Offsets'.format(snap)][ffT].astype('int')
        mstar = sim['SnapNumber_{}/PartType4/Masses'.format(snap)][offMain[0]:offMain[1]]
        Mstar = np.sum(mstar)
        
        suc2 = successors[1:]
        mothe = 0
        for s in suc2:
            snapT, iddC = split_unique_id(int(s))
            offSat = sim['SnapNumber_{}/SubGroups/PartType4/Offsets'.format(snap)][iddC].astype('int')
            mstar = sim['SnapNumber_{}/PartType4/Masses'.format(snap)][offSat[0]:offSat[1]]
            mothe = mothe+np.sum(mstar)
            
        mrat = mothe/Mstar
        Mergers.append(mrat)
        
Mergers, distance = np.array(Mergers), np.array(distance)

In [15]:
for m, d in zip(Mergers, distance):
    print(m, d)

0.0 1.3794043
0.0 1.8595378
0.0 1.9519744
0.0 2.1699753
0.0 2.4851022
0.0 2.742187
0.0 4.048925
0.0 4.2912803
0.11172172231479213 6.179609
0.19907985465490188 6.4122596
0.0 6.836518
0.05714562055867132 6.976785
0.0015232544038176986 7.799981
0.0 8.033096
0.0 9.168441
0.0 9.796904
0.0 9.808259
0.0 10.134477
0.0 10.706467
0.0 11.499826
0.0 16.507261


In [16]:
def getMergers(ff, cc):
    uid = int(127*1e6+ff)
    stree = networkx.dfs_tree(trees, str(uid))

    uidC = int(127*1e6+cc)
    streeC = networkx.dfs_tree(trees, str(uidC))

    Mergers, distance, zmerger, iddm, snapm = [], [], [], [], []

    while True:
        successors = list(stree.successors(str(uid)))
        successorsC = list(streeC.successors(str(uidC)))
        if len(successors) == 0 or len(successorsC) == 0:
            break
        uid = successors[0]
        uidC = successorsC[0]
        if len(successors)>1: # that is, a mergers happens
            snap, ffT = split_unique_id(int(uid))
            snapC, ccT = split_unique_id(int(uidC))

            gro = sim['SnapNumber_{}/SubGroups/GroupNumber'.format(snap)][ccT]
            R200 = sim['SnapNumber_{}/Groups/Group_R_Crit200'.format(snap)][gro]

            cen = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][ccT]
            gal = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][ffT]

            
            zz = sim['SnapNumber_{}/Header/Redshift'.format(snap)][()]
            dist = np.sqrt(np.sum((cen-gal)**2))/R200

            distance.append(dist)

            offMain = sim['SnapNumber_{}/SubGroups/PartType4/Offsets'.format(snap)][ffT].astype('int')
            mstar = sim['SnapNumber_{}/PartType4/Masses'.format(snap)][offMain[0]:offMain[1]]
            Mstar = np.sum(mstar)

            suc2 = successors[1:]
            mothe = 0
            idt = []
            for s in suc2:
                snapT, iddC = split_unique_id(int(s))
                offSat = sim['SnapNumber_{}/SubGroups/PartType4/Offsets'.format(snap)][iddC].astype('int')
                mstar = sim['SnapNumber_{}/PartType4/Masses'.format(snap)][offSat[0]:offSat[1]]
                mothe = mothe+np.sum(mstar)
                idt.append(iddC)

            mrat = mothe/Mstar
            Mergers.append(mrat)
            zmerger.append(zz)
            snapm.append(snap)
            iddm.append(idt)

    return np.array(Mergers), np.array(distance), np.array(zmerger), np.array(snapm), np.array(iddm)

In [17]:
Mergers, distance, zs, snapm, idsm = getMergers(4470, 4469)

In [18]:
for m, d, zi in zip(Mergers, distance, zs):
    print(m, d, zs)

0.010122022388020395 0.73242867 [0.02345296 0.04768037 0.12548675 0.19704373 0.21218879 0.25947302
 0.27588908 0.29265319 0.30977887 0.34517284 0.36347225 0.38219559
 0.40136085 0.42098718 0.44109488 0.46170561 0.4828424  0.50452986
 0.5267942  0.54966348 0.57316772 0.59733906 0.62221201 0.64782359
 0.67421368 0.7014252  0.72950446 0.75850151 0.78847044 0.8848215
 0.91931835 0.95513702 0.99236787 1.03111004 1.07147251 1.11357545
 1.15755164 1.20354826 1.25172889 1.30227591 1.35539334 1.47028462
 1.53260841 1.59861325 1.66867764 1.74323563 1.82278768 1.90791418
 1.99929222 2.0977173  2.20413083 2.31965588 2.44564445 2.58374016
 2.73596329 2.904828   3.09138587 3.28780956 3.49330299 3.70831591
 3.93331747 4.16879764 4.67326355 4.94334262 5.22608943 5.5221147
 5.83205712 6.15658485 6.4963967  6.85222418 7.22483254 7.61502281
 8.45154143]
0.0 0.8304049 [0.02345296 0.04768037 0.12548675 0.19704373 0.21218879 0.25947302
 0.27588908 0.29265319 0.30977887 0.34517284 0.36347225 0.38219559
 0.40

In [19]:
# ok, for all, and keep the most important merger, and then select and show the ratio of thsi mergers, 2r200 i think is ok

mmerger, zzmerger = [], []
mmsnap, mmids = [], []

for s, c in zip(Satellites, Centrals):
    Mergers, distance, zs, snp, idds = getMergers(s, c)
    i = np.where(distance<2.)[0]
    if i.size>1:
        mm = Mergers[i]
        zz = zs[i]
        snpz = snp[i]
        iddsz = idds[i]
        mmerger.append(mm.max())
        kk = np.where(mm==mm.max())[0][0]
        zzmerger.append(zz[kk])
        mmsnap.append(snpz[kk])
        mmids.append(iddsz[kk])
    else:
        mmerger.append(0.)
        zzmerger.append(0.)
        mmsnap.append(0)
        mmids.append(0.)
        
mmerger = np.array(mmerger)
zzmerger = np.array(zzmerger)
mmsnap = np.array(mmsnap)
mmids = np.array(mmids)

/home/srodriguez/miniconda3/envs/cielo_old/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
print('IDs\tCent\tMergers\tcosCen\t\t\tstMass\t\t\tMu\t\t\tzmerger')
for i in range(Satellites.size):
    print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(Satellites[i], Centrals[i], mmerger[i], CosCen[i], StMass[i], Mu[i], zzmerger[i]))

IDs	Cent	Mergers	cosCen			stMass			Mu			zmerger
152	151	0.0	0.0	7.69437017371166	0.19028741328047571	0.0
4338	4337	0.0	0.0	8.824296879623581	0.5248884311706145	0.2926531855399035
4339	4337	0.0	0.0	8.395722114289644	2.086676217765043	0.647823587803394
4341	4337	0.0	0.0	8.237350760201767	2.198490566037736	0.0
4343	4337	0.0	0.0	8.100917132543179	0.0	0.0
4470	4469	0.550081080322482	0.0	9.557141203991026	0.446042746491177	0.38219559019532867
4471	4469	0.00015900841628144143	0.0	9.561488709317695	3.0531650071123755	0.420987180617459
4473	4469	0.0015126735222379475	0.0	8.581329133791673	4.319748427672956	0.06009857608308211
4474	4469	0.0	0.0	8.71543015412092	11.480499219968799	0.0
4476	4469	0.0	0.0	8.267004492441973	25.38836865450961	0.0
4478	4469	0.0	0.0	8.021852922806016	0.0	0.0


In [21]:
group = sim['SnapNumber_128/SubGroups/GroupNumber'][4341]
GMass = sim['SnapNumber_128/Groups/Group_M_Crit200'][group]
np.log10(GMass*1e10)

11.97874485031806

In [22]:
group = sim['SnapNumber_128/SubGroups/GroupNumber'][4476]
GMass = sim['SnapNumber_128/Groups/Group_M_Crit200'][group]
np.log10(GMass*1e10)

11.77684956367362

In [23]:
# mass at z of entry.

StMassZ = []
GasMassZ = []

for sn, idd, cen in zip(Snap, idSnap, cenSnap):
    Offstar = sim['SnapNumber_{}/SubGroups/PartType4/Offsets'.format(sn)][idd].astype('int')
    MassStar = sim['SnapNumber_{}/PartType4/Masses'.format(sn)][Offstar[0]:Offstar[1]]
    
    StMassZ.append(np.log10(np.sum(MassStar)*1e10))
    
    Offgas = sim['SnapNumber_{}/SubGroups/PartType0/Offsets'.format(sn)][idd].astype('int')
    MassGas = sim['SnapNumber_{}/PartType0/Masses'.format(sn)][Offgas[0]:Offgas[1]]
    
    GasMassZ.append(np.log10(np.sum(MassGas)*1e10))
     
StMassZ, GasMassZ = np.array(StMassZ), np.array(GasMassZ)

In [24]:
# gas mass at z=0, i miss that

GasMass = []

for s in Satellites:
    Offgas = sim['SnapNumber_128/SubGroups/PartType0/Offsets'][s].astype('int')
    MassGas = sim['SnapNumber_128/PartType0/Masses'][Offgas[0]:Offgas[1]]
    
    GasMass.append(np.log10(np.sum(MassGas)*1e10))
    
GasMass = np.array(GasMass)

/home/srodriguez/miniconda3/envs/cielo_old/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':


In [25]:
print('IDs\tStMass\tStMass at entry\tGas Mass\tGas Mass at entry')
for i in range(Satellites.size):
    print('{}\t{}\t{}\t{}\t{}'.format(Satellites[i], StMass[i], StMassZ[i], GasMass[i], GasMassZ[i]))

IDs	StMass	StMass at entry	Gas Mass	Gas Mass at entry
152	7.69437017371166	7.694370214589153	7.879736245824006	7.879736245824006
4338	8.824296879623581	8.66286934593257	8.434503624899342	9.260392701420436
4339	8.395722114289644	8.26874956995319	8.206891707259528	9.0089800390066
4341	8.237350760201767	8.086709215227403	7.92796141988879	8.897223274518744
4343	8.100917132543179	8.444391820381707	-inf	8.656937892384564
4470	9.557141203991026	9.554577103026954	9.817616629735054	9.825865859636755
4471	9.561488709317695	9.304332692840317	9.478072789305148	10.098723045555818
4473	8.581329133791673	8.548844135522156	8.825461424019583	8.879029620318681
4474	8.71543015412092	8.522731241401026	9.142521984518371	9.396247760098511
4476	8.267004492441973	8.267004492441973	7.088296737085916	7.084350660490255
4478	8.021852922806016	8.021852922806016	8.009592814794582	8.009592814794582


In [26]:
print('IDs\tStMass/stMassthen\tGasMass/GasMassThen')
for i in range(Satellites.size):
    print('{}\t{}\t{}'.format(Satellites[i], 10**StMass[i]/10**StMassZ[i], 10**GasMass[i]/10**GasMassZ[i]))

IDs	StMass/stMassthen	GasMass/GasMassThen
152	0.9999999058760995	1.0
4338	1.4501987741759856	0.1493175734012346
4339	1.3395919970803367	0.15772904298204063
4341	1.4146257038321253	0.10733420535872258
4343	0.45344572537002575	0.0
4470	1.0059215239741697	0.9811847059560701
4471	1.8078234540655176	0.23952439051491517
4473	1.0776680274222903	0.8839583522825497
4474	1.5584716723999736	0.5575376810753022
4476	1.0	1.0091275817607497
4478	1.0	1.0


In [27]:
print('IDs\tMergers\tzmerger\tsnap\tidss')
for i in range(Satellites.size):
    print('{}\t{}\t{}\t{}\t{}'.format(Satellites[i], mmerger[i], zzmerger[i], mmsnap[i], mmids[i]))

IDs	Mergers	zmerger	snap	idss
152	0.0	0.0	0	0.0
4338	0.0	0.2926531855399035	106	[2402]
4339	0.0	0.647823587803394	89	[5326]
4341	0.0	0.0	0	0.0
4343	0.0	0.0	0	0.0
4470	0.550081080322482	0.38219559019532867	101	[2495]
4471	0.00015900841628144143	0.420987180617459	99	[2618]
4473	0.0015126735222379475	0.06009857608308211	122	[4675]
4474	0.0	0.0	0	0.0
4476	0.0	0.0	0	0.0
4478	0.0	0.0	0	0.0


In [28]:
sim['SnapNumber_128/PartType0/'].keys()

<KeysViewHDF5 ['Abundances', 'BindingEnergy', 'Circularity', 'Coordinates', 'Density', 'ElectronAbundance', 'Esn', 'EsnCold', 'GroupNumber', 'InternalEnergy', 'Masses', 'NeutralHydrogenAbundance', 'ParticleIDs', 'Potential', 'SmoothingLength', 'SpecificAngularMomentum', 'StarFormationRate', 'SubFindNumber', 'SubGroupNumber', 'Velocities']>

In [29]:
import matplotlib.cm as mmc

In [30]:
mmc.viridis(0.5)

(0.127568, 0.566949, 0.550556, 1.0)

In [31]:
norm = mpc.Normalize(vmin=-3.,vmax=0.5)

In [32]:
norm(-0.5)

0.7142857142857143

In [33]:
mmc.viridis(norm(-0.5))

(0.288921, 0.758394, 0.428426, 1.0)

In [34]:
StMass

array([7.69437017, 8.82429688, 8.39572211, 8.23735076, 8.10091713,
       9.5571412 , 9.56148871, 8.58132913, 8.71543015, 8.26700449,
       8.02185292])

In [35]:
np.abs(CosCen)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
M200G = []
for c in Centrals:
    group = sim['SnapNumber_128/SubGroups/GroupNumber'][c]
    GMass = sim['SnapNumber_128/Groups/Group_M_Crit200'][group]
    M200G.append(np.log10(GMass*1e10))
    
M200G = np.array(M200G)

In [37]:
print('SubGroup & Stellar Mass & $\cos \theta_i$ & Merger ratio & Host Group Mass & Stellar Mass ratio & Gas Mass ratio\\\hline')

for i in range(Satellites.size):
    print('{} & {}/$\\log(M_\\odot)$ & {} & {} & {}/$\log(M_\\odot)$ & {} & {}\\\\'.format(Satellites[i], StMass[i], np.abs(CosCen[i]), mmerger[i], M200G[i], 10**StMass[i]/10**StMassZ[i], 10**GasMass[i]/10**GasMassZ[i]))

SubGroup & Stellar Mass & $\cos 	heta_i$ & Merger ratio & Host Group Mass & Stellar Mass ratio & Gas Mass ratio\\hline
152 & 7.69437017371166/$\log(M_\odot)$ & 0.0 & 0.0 & 10.110525800565949/$\log(M_\odot)$ & 0.9999999058760995 & 1.0\\
4338 & 8.824296879623581/$\log(M_\odot)$ & 0.0 & 0.0 & 11.97874485031806/$\log(M_\odot)$ & 1.4501987741759856 & 0.1493175734012346\\
4339 & 8.395722114289644/$\log(M_\odot)$ & 0.0 & 0.0 & 11.97874485031806/$\log(M_\odot)$ & 1.3395919970803367 & 0.15772904298204063\\
4341 & 8.237350760201767/$\log(M_\odot)$ & 0.0 & 0.0 & 11.97874485031806/$\log(M_\odot)$ & 1.4146257038321253 & 0.10733420535872258\\
4343 & 8.100917132543179/$\log(M_\odot)$ & 0.0 & 0.0 & 11.97874485031806/$\log(M_\odot)$ & 0.45344572537002575 & 0.0\\
4470 & 9.557141203991026/$\log(M_\odot)$ & 0.0 & 0.550081080322482 & 11.77684956367362/$\log(M_\odot)$ & 1.0059215239741697 & 0.9811847059560701\\
4471 & 9.561488709317695/$\log(M_\odot)$ & 0.0 & 0.00015900841628144143 & 11.77684956367362/$\log